## Summary files for *"Bundling Measures for Food Systems Transformation: a global, multimodel assessment"*

This Jupyter Notebook generates Excel sheets that summarize the dataset.

In [1]:
import sys
import pandas as pd

import json
import pickle
import os
from os.path import join as pjoin
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

sys.path.append("..")
import applepy as apy
from applepy.utils.helper import *
from applepy.utils.calculations import decomposition,basic

# pd.set_option('display.max_rows', None)

## Load Datasets

In [2]:
fp = '../data/global-paper_dataset.csv'

df = pd.read_csv(fp,index_col=0)#.drop(columns=['index'])
df.head()

,model,scenario,region,variable,item,unit,year,value,BAU_ref_year,percent_change_BAU_ref_year,diff_BAU_ref_year,percent_change_BAU,diff_BAU,percent_change_ELM,diff_ELM
0,GLOBIOM,BAU,WLD,ANNR_prod,TOT,Absolute number,2020.0,1.515144e+09,2020.0,0.000000,0.0,0.000000,0.0,0.200102,3025784.0
1,GLOBIOM,BAU,WLD,ANNR_prod,TOT,Absolute number,2050.0,1.841336e+09,2020.0,21.528765,326191883.0,0.000000,0.0,64.673304,723160941.0
2,GLOBIOM,BAU_DIET,WLD,ANNR_prod,TOT,Absolute number,2020.0,1.515144e+09,2020.0,0.000000,0.0,0.000000,0.0,0.200102,3025784.0
3,GLOBIOM,BAU_DIET,WLD,ANNR_prod,TOT,Absolute number,2050.0,1.281021e+09,2020.0,-15.452194,-234123055.0,-30.429799,-560314938.0,14.563548,162846003.0
4,GLOBIOM,BAU_MITI,WLD,ANNR_prod,TOT,Absolute number,2020.0,1.512119e+09,2020.0,-0.199703,-3025784.0,-0.199703,-3025784.0,0.000000,0.0


In [3]:
fp_dc = '../data/global-paper_decomposition.csv'
df_dc = pd.read_csv(fp_dc,index_col=0)#.drop(columns=['index'])
df_dc.head()

base_filename_dc = fp_dc.split('/')[-1].split('.csv')[0]
base_filename_dc

'global-paper_decomposition'

In [4]:
base_filename = fp.split('/')[-1].split('.csv')[0]
output_dir = pjoin(os.getcwd(),'output','paper-tables',base_filename)
check_path(output_dir)

created directory: /Users/mms466/Documents/fsgc/dev/EAT-Lancet-2_modelling_data-processing-viz/jupyter-notebooks/output/paper-tables/global-paper_dataset


## Food Systems Indicators

In [5]:
# Input parameters
variables = ['CONS','FOOD','PROD','CALI','CALO','AREA','YILD','XPRP']
scenarios = ['BAU','BAU_PROD','BAU_WAST','BAU_DIET','EL2','BAU_MITI','ELM']
regions = ['WLD']
items_list = ['AGR', 'CRP','LSP','VFN','RUM']  # List of items to iterate through
year = 2050
cols = ['model', 'scenario', 'region', 'variable', 'item', 'unit', 'percent_change_BAU_ref_year', 'percent_change_BAU']

# Create a new Excel writer to save multiple sheets in a single file
with pd.ExcelWriter(pjoin(output_dir, f"{base_filename}_summary.xlsx")) as writer:
    for item in items_list:
        # Filter data based on criteria for each item
        fdf = df[
            (df.scenario.isin(scenarios)) &
            (df.region.isin(regions)) &
            (df.variable.isin(variables)) &
            (df.item == item) &
            (df.year == year)
        ][cols]

        # Pivot table for 'percent_change_BAU'
        fdf_p = fdf.pivot_table(index=['scenario', 'model'], columns='variable', values='percent_change_BAU')
        fdf_p_describe = fdf_p.groupby(['scenario']).describe().reindex(scenarios)
        result_BAU = fdf_p_describe.sort_index(axis=1, level=0).loc[slice(None), (variables, ['50%', 'min', 'max', 'count'])]
        
        # Pivot table for 'percent_change_BAU_ref_year'
        fdf_p_ref = fdf.pivot_table(index=['scenario', 'model'], columns='variable', values='percent_change_BAU_ref_year')
        fdf_p_ref_describe = fdf_p_ref.groupby(['scenario']).describe().reindex(scenarios)
        result_BAU_ref = fdf_p_ref_describe.sort_index(axis=1, level=0).loc[slice(None), (variables, ['50%', 'min', 'max', 'count'])]
        
        # Save each result as a separate sheet
        result_BAU.to_excel(writer, sheet_name=f"{item}_percent_change_BAU")
        result_BAU_ref.to_excel(writer, sheet_name=f"{item}_percent_change_BAU_ref_year")

print("Excel file with multiple sheets created successfully.")


Excel file with multiple sheets created successfully.


In [6]:
# Input parameters
variables = ['ECH4','EN2O','ECO2','WATR','FRTN','FRTP']
scenarios = ['BAU','BAU_PROD','BAU_WAST','BAU_DIET','EL2','BAU_MITI','ELM']
regions = ['WLD']
items_list = ['AGR', 'CRP','LSP','VFN','RUM']  # List of items to iterate through
year = 2050
cols = ['model', 'scenario', 'region', 'variable', 'item', 'unit', 'percent_change_BAU_ref_year', 'percent_change_BAU']


# Load your DataFrame here (replace this with actual data loading if needed)
# df = pd.read_csv("path_to_your_data.csv")

# Create a new Excel writer to save multiple sheets in a single file
with pd.ExcelWriter(pjoin(output_dir, f"{base_filename}_env-summary.xlsx")) as writer:
    for item in items_list:
        try:
            # Filter data based on criteria for each item
            fdf = df[
                (df.scenario.isin(scenarios)) &
                (df.region.isin(regions)) &
                (df.variable.isin(variables)) &
                (df.item == item) &
                (df.year == year)
            ][cols]

            # Pivot table for 'percent_change_BAU'
            fdf_p = fdf.pivot_table(index=['scenario', 'model'], columns='variable', values='percent_change_BAU')
            fdf_p_describe = fdf_p.groupby(['scenario']).describe().reindex(scenarios)
            result_BAU = fdf_p_describe.sort_index(axis=1, level=0).loc[slice(None), (variables, ['50%', 'min', 'max', 'count'])]
            
            # Pivot table for 'percent_change_BAU_ref_year'
            fdf_p_ref = fdf.pivot_table(index=['scenario', 'model'], columns='variable', values='percent_change_BAU_ref_year')
            fdf_p_ref_describe = fdf_p_ref.groupby(['scenario']).describe().reindex(scenarios)
            result_BAU_ref = fdf_p_ref_describe.sort_index(axis=1, level=0).loc[slice(None), (variables, ['50%', 'min', 'max', 'count'])]
            
            # Save each result as a separate sheet
            result_BAU.to_excel(writer, sheet_name=f"{item}_percent_change_BAU")
            result_BAU_ref.to_excel(writer, sheet_name=f"{item}_percent_change_BAU_ref_year")
        except Exception as e:
            print(e, item)

print("Excel file with multiple sheets created successfully.")


'WATR' LSP
'WATR' RUM
Excel file with multiple sheets created successfully.


In [7]:
# Input parameters
variables = ['LAND_added']
scenarios = ['BAU','BAU_PROD','BAU_WAST','BAU_DIET','EL2','BAU_MITI','ELM']
regions = ['WLD']
items_list = ['AGR_added','CRP','GRS','ONV_added','LAND_tot']  # List of items to iterate through
year = 2050
cols = ['model', 'scenario', 'region', 'variable', 'item', 'unit', 'percent_change_BAU_ref_year', 'percent_change_BAU']


# Create a new Excel writer to save multiple sheets in a single file
with pd.ExcelWriter(pjoin(output_dir, f"{base_filename}_land-summary.xlsx")) as writer:
    for item in items_list:
        # Filter data based on criteria for each item
        fdf = df[
            (df.scenario.isin(scenarios)) &
            (df.region.isin(regions)) &
            (df.variable.isin(variables)) &
            (df.item == item) &
            (df.year == year)
        ][cols]

        # Pivot table for 'percent_change_BAU'
        fdf_p = fdf.pivot_table(index=['scenario', 'model'], columns='variable', values='percent_change_BAU')
        fdf_p_describe = fdf_p.groupby(['scenario']).describe().reindex(scenarios)
        result_BAU = fdf_p_describe.sort_index(axis=1, level=0).loc[slice(None), (variables, ['50%', 'min', 'max', 'count'])]
        
        # Pivot table for 'percent_change_BAU_ref_year'
        fdf_p_ref = fdf.pivot_table(index=['scenario', 'model'], columns='variable', values='percent_change_BAU_ref_year')
        fdf_p_ref_describe = fdf_p_ref.groupby(['scenario']).describe().reindex(scenarios)
        result_BAU_ref = fdf_p_ref_describe.sort_index(axis=1, level=0).loc[slice(None), (variables, ['50%', 'min', 'max', 'count'])]
        
        # Save each result as a separate sheet
        result_BAU.to_excel(writer, sheet_name=f"{item}_percent_change_BAU")
        result_BAU_ref.to_excel(writer, sheet_name=f"{item}_percent_change_BAU_ref_year")

print("Excel file with multiple sheets created successfully.")


Excel file with multiple sheets created successfully.


/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [8]:
# Input parameters
variables = ['EMIS_nonCO2']
scenarios = ['BAU','BAU_PROD','BAU_WAST','BAU_DIET','EL2','BAU_MITI','ELM']
regions = ['WLD']
items_list = ['AGR']  # List of items to iterate through
year = 2050
cols = ['model', 'scenario', 'region', 'variable', 'item', 'unit', 'percent_change_BAU_ref_year', 'percent_change_BAU']


# Create a new Excel writer to save multiple sheets in a single file
with pd.ExcelWriter(pjoin(output_dir, f"{base_filename}_nonCO2-emis-summary.xlsx")) as writer:
    for item in items_list:
        # Filter data based on criteria for each item
        fdf = df[
            (df.scenario.isin(scenarios)) &
            (df.region.isin(regions)) &
            (df.variable.isin(variables)) &
            (df.item == item) &
            (df.year == year)
        ][cols]

        # Pivot table for 'percent_change_BAU'
        fdf_p = fdf.pivot_table(index=['scenario', 'model'], columns='variable', values='percent_change_BAU')
        fdf_p_describe = fdf_p.groupby(['scenario']).describe().reindex(scenarios)
        result_BAU = fdf_p_describe.sort_index(axis=1, level=0).loc[slice(None), (variables, ['50%', 'min', 'max', 'count'])]
        
        # Pivot table for 'percent_change_BAU_ref_year'
        fdf_p_ref = fdf.pivot_table(index=['scenario', 'model'], columns='variable', values='percent_change_BAU_ref_year')
        fdf_p_ref_describe = fdf_p_ref.groupby(['scenario']).describe().reindex(scenarios)
        result_BAU_ref = fdf_p_ref_describe.sort_index(axis=1, level=0).loc[slice(None), (variables, ['50%', 'min', 'max', 'count'])]
        
        # Save each result as a separate sheet
        result_BAU.to_excel(writer, sheet_name=f"{item}_percent_change_BAU")
        result_BAU_ref.to_excel(writer, sheet_name=f"{item}_percent_change_BAU_ref_year")

print("Excel file with multiple sheets created successfully.")


Excel file with multiple sheets created successfully.


In [9]:
variables =['GDPT','POPT']
scenarios = ['BAU','BAU_PROD','BAU_WAST','BAU_DIET','EL2','BAU_MITI','ELM']
regions = ['WLD']
items = ['TOT']
year = 2050
cols =  ['model','scenario','region','variable','item','unit','percent_change_BAU_ref_year', 'percent_change_BAU']
fdf = df[(df.scenario.isin(scenarios)) &
        (df.region.isin(regions))&
        (df.variable.isin(variables)) &
        (df.item.isin(items)) &
        (df.year==year)][cols]
fdf_p = fdf.pivot_table(index=['scenario','model'],columns='variable',values = 'percent_change_BAU_ref_year')
fdf_p_describe = fdf_p.groupby(['scenario']).describe().reindex(scenarios)
fdf_p_describe.sort_index(axis=1, level=0).loc[slice(None),(variables,['50%','min','max','count'])].to_excel(pjoin(output_dir,base_filename+'_demographics-summary.xlsx.xlsx'))

## Decomposition Analysis Summary

In [10]:

sheet_sets ={ 'fs_indicators': {'variables':['CONS','FOOD','PROD','CALI','CALO','AREA','YILD','XPRP'],
                                'items': ['AGR', 'CRP','LSP','RUM','VFN']},
                'env_indicators': {'variables':['ECH4','EN2O','WATR','FRTN','FRTP'],
                                'items': ['AGR', 'CRP']},
                'env_indicators_emis': {'variables':['EMIS_nonCO2',],
                                'items': ['AGR']},
                'env_indicators_land': {'variables':['LAND_added'],
                                'items': ['AGR_added', 'CRP','GRS']},
                                }


scenarios = ['BAU','BAU_PROD','BAU_WAST','BAU_DIET','EL2','BAU_MITI','ELM']
regions = ['WLD']
items_list = []  # List of items to iterate through
year = 2050
cols = ['individual', 'total', 'interaction', 'model', 'region', 'variable',
       'item', 'year', 'unit', 'driver']

value_type = 'percent_change_BAU'
normalized = False

# Create a new Excel writer to save multiple sheets in a single file
with pd.ExcelWriter(pjoin(output_dir, f"{base_filename_dc}_summary.xlsx")) as writer:
    for sheet_set in sheet_sets.keys():
        variables = sheet_sets[sheet_set]['variables']
        items_list = sheet_sets[sheet_set]['items']

        for item in items_list:
            print(variables,item)
        # Filter data based on criteria for each item
        
            fdf = df_dc[(df_dc.region.isin(regions)) &
                    (df_dc.variable.isin(variables)) &
                    (df_dc.item == item) &
                    (df_dc.normalized==normalized) &
                    (df_dc.value_type==value_type) & 
                    (df_dc.year==year)][cols]

            df_dc_l = fdf.melt(id_vars=['model','region','variable','driver'],value_vars=['total','individual','interaction'],var_name='effect')
            df_dc_l_p = df_dc_l.pivot_table(index=['region','model','driver','effect'], columns='variable',values='value').reset_index()
            df_dc_l_p_describe = df_dc_l_p.groupby(['region','driver','effect']).describe()
            df_dc_l_p.groupby(['region','driver','effect']).describe().loc[slice(None),(variables,['50%','min','max','count'])].to_excel(writer, sheet_name=f"{sheet_set}_{item}_effects-summary")

            df_dc_l = fdf.melt(id_vars=['model','region','variable','driver'],value_vars=['total','individual','interaction'],var_name='effect')
            df_dc_l_p = df_dc_l.pivot_table(index=['model','driver','effect'], columns='variable',values='value').reset_index()
            df_dc_l_p.groupby(['driver','model','effect']).describe().loc[slice(None),(variables,['50%'])].to_excel(writer, sheet_name=f"{sheet_set}_{item}_effects-models")

print("Excel file with multiple sheets created successfully.")


/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


['CONS', 'FOOD', 'PROD', 'CALI', 'CALO', 'AREA', 'YILD', 'XPRP'] AGR


/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


['CONS', 'FOOD', 'PROD', 'CALI', 'CALO', 'AREA', 'YILD', 'XPRP'] CRP


/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


...

/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


['ECH4', 'EN2O', 'WATR', 'FRTN', 'FRTP'] CRP


/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/py

['EMIS_nonCO2'] AGR
['LAND_added'] AGR_added
['LAND_added'] CRP
['LAND_added'] GRS
Excel file with multiple sheets created successfully.


/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/Users/mms466/anaconda3/envs/el-modelling_v4-test-1/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
